In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from datetime import datetime

In [2]:
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')

In [3]:
spark = SparkSession.builder\
    .appName('austism-and-green-space')\
    .config(conf=conf)\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6d257f90-2bf6-44c2-b875-0b0b4acd251f;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 280ms :: artifacts dl 7ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------------------

22/08/06 00:40:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/06 00:40:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.csv("s3a://projeto-pi-datalake-raw/autism and green space_clean.csv", header=True)

22/08/06 00:41:12 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
execution_date = datetime(2022, 5, 29)

In [6]:
trusted_df = (
    df
    .select(
        "Autism", f.col("LC11").alias("Water_Area"), 
        f.col("LC21").alias("Open_Land"), f.col("LC2B").alias("Urban_Land"),
        f.col("LC31").alias("Barren_Land"), f.col("LC4").alias("Forest_Land"),
        f.col("LC52").alias("Shrub_Land"), f.col("LC71").alias("Grass_Land"),
        f.col("LC8").alias("Agro_Land"), f.col("LC9").alias("Wet_Land"),
        "Pop", "Income_median"
    )
    .withColumn('year', f.lit(execution_date.year))
    .withColumn('month', f.lit(execution_date.month))
    .withColumn('day', f.lit(execution_date.day))
)

In [7]:
trusted_df.write.mode("overwrite").partitionBy("year", "month", "day")\
    .parquet("s3a://projeto-pi-datalake-trusted/autism-greespace")